In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import pandas as pd
import datetime

class ExpenseTracker:
    def __init__(self):
        # Initialize an empty DataFrame with columns: Date, Amount, Category, Subcategory, Description
        self.df = pd.DataFrame(columns=["Date", "Amount", "Category", "Subcategory", "Description"])

        # Predefined categories and their subcategories
        self.categories = {
            "Food": ["Groceries", "Dining Out", "Takeout"],
            "Transportation": ["Fuel", "Public Transport", "Taxis"],
            "Utilities": ["Electricity", "Water", "Internet"],
            "Entertainment": ["Movies", "Concerts", "Games"],
            "Health": ["Medicines", "Doctor Visits", "Gym"],
            "Shopping": ["Clothing", "Electronics", "Accessories"],
            "Others": ["Miscellaneous"]
        }

    def add_expense(self, amount, category, subcategory, description=""):
        """Add an expense record."""
        date = datetime.datetime.now()  # Current date and time as a datetime object
        new_expense = pd.DataFrame([[date, amount, category, subcategory, description]], columns=self.df.columns)

        # Ensure that no empty columns are included
        new_expense = new_expense.dropna(axis=1, how='all')  # Drop columns with all NaN values if any

        # Concatenate the DataFrame to include the new expense entry
        self.df = pd.concat([self.df, new_expense], ignore_index=True)

    def generate_report(self):
        """Generate a report for total expenses and categorized breakdown."""
        total_expenses = self.df['Amount'].sum()
        category_report = self.df.groupby(['Category', 'Subcategory'])['Amount'].sum().reset_index()
        monthly_summary = self.df.groupby(self.df['Date'].dt.to_period('M')).agg({'Amount': 'sum'}).reset_index()
        
        return total_expenses, category_report, monthly_summary


class ExpenseTrackerGUI:
    def __init__(self, root, tracker):
        self.root = root
        self.tracker = tracker
        self.root.title("Expense Tracker")

        # Set light background for the root window
        self.root.configure(bg="#F5F5F5")

        # Create a frame for adding expenses
        self.frame_add_expense = tk.Frame(self.root, bg="#F5F5F5")
        self.frame_add_expense.pack(pady=10)

        # Labels and entry fields for adding an expense with light theme
        self.label_amount = tk.Label(self.frame_add_expense, text="Amount:", bg="#F5F5F5", fg="black")
        self.label_amount.grid(row=0, column=0)
        self.entry_amount = tk.Entry(self.frame_add_expense, bg="#FFFFFF", fg="black", insertbackground="black")
        self.entry_amount.grid(row=0, column=1)

        self.label_category = tk.Label(self.frame_add_expense, text="Category:", bg="#F5F5F5", fg="black")
        self.label_category.grid(row=1, column=0)
        
        # OptionMenu for selecting category with light theme
        self.selected_category = tk.StringVar()
        self.selected_category.set(list(self.tracker.categories.keys())[0])  # Default category
        
        self.category_menu = tk.OptionMenu(self.frame_add_expense, self.selected_category, *self.tracker.categories.keys(), command=self.update_subcategories)
        self.category_menu.grid(row=1, column=1)
        self.category_menu.configure(bg="#FFFFFF", fg="black")

        self.label_subcategory = tk.Label(self.frame_add_expense, text="Subcategory:", bg="#F5F5F5", fg="black")
        self.label_subcategory.grid(row=2, column=0)
        
        # OptionMenu for selecting subcategory with light theme
        self.selected_subcategory = tk.StringVar()
        self.selected_subcategory.set("")  # Default empty subcategory
        self.subcategory_menu = tk.OptionMenu(self.frame_add_expense, self.selected_subcategory, "")
        self.subcategory_menu.grid(row=2, column=1)
        self.subcategory_menu.configure(bg="#FFFFFF", fg="black")

        self.label_description = tk.Label(self.frame_add_expense, text="Description:", bg="#F5F5F5", fg="black")
        self.label_description.grid(row=3, column=0)
        self.entry_description = tk.Entry(self.frame_add_expense, bg="#FFFFFF", fg="black", insertbackground="black")
        self.entry_description.grid(row=3, column=1)

        # Add Expense button with light theme
        self.btn_add_expense = tk.Button(self.frame_add_expense, text="Add Expense", command=self.add_expense, bg="#4CAF50", fg="white")
        self.btn_add_expense.grid(row=4, columnspan=2)

        # Frame for expense report and treeview display
        self.frame_report = tk.Frame(self.root, bg="#F5F5F5")
        self.frame_report.pack(pady=10)

        # Treeview for showing expenses with light theme
        self.tree = ttk.Treeview(self.frame_report, columns=("Date", "Amount", "Category", "Subcategory", "Description"), show="headings")
        self.tree.heading("Date", text="Date")
        self.tree.heading("Amount", text="Amount")
        self.tree.heading("Category", text="Category")
        self.tree.heading("Subcategory", text="Subcategory")
        self.tree.heading("Description", text="Description")
        
        # Set column widths
        self.tree.column("Date", width=150)
        self.tree.column("Amount", width=100)
        self.tree.column("Category", width=100)
        self.tree.column("Subcategory", width=100)
        self.tree.column("Description", width=200)

        # Create a light theme for Treeview using ttk.Style
        style = ttk.Style()
        style.configure("Light.Treeview",
                        background="#FFFFFF",
                        foreground="black",
                        fieldbackground="#FFFFFF",
                        rowheight=25)
        
        # Apply the style to the Treeview
        self.tree.tag_configure("Light", background="#FFFFFF", foreground="black")
        self.tree.configure(style="Light.Treeview")

        self.tree.pack()

        # Generate Report button with light theme
        self.btn_generate_report = tk.Button(self.root, text="Generate Report", command=self.generate_report, bg="#2196F3", fg="white")
        self.btn_generate_report.pack(pady=5)

        # View All Expenses button with light theme
        self.btn_view_expenses = tk.Button(self.root, text="View All Expenses", command=self.view_expenses, bg="#FFEB3B", fg="black")
        self.btn_view_expenses.pack(pady=5)

    def update_subcategories(self, category):
        """Update the subcategory dropdown based on the selected category."""
        subcategories = self.tracker.categories.get(category, [])
        self.selected_subcategory.set(subcategories[0] if subcategories else "")
        menu = self.subcategory_menu["menu"]
        menu.delete(0, "end")
        for subcategory in subcategories:
            menu.add_command(label=subcategory, command=tk._setit(self.selected_subcategory, subcategory))

    def add_expense(self):
        """Add an expense record from the input fields."""
        try:
            # Get the amount as float
            amount = float(self.entry_amount.get())  
            
            # Get the selected category and subcategory
            category = self.selected_category.get()  
            subcategory = self.selected_subcategory.get() 
            
            # Get description (optional)
            description = self.entry_description.get()
            
            # Ensure category and subcategory are selected
            if not category or not subcategory:
                raise ValueError("Category and Subcategory cannot be empty")
            
            # Add the expense to the tracker
            self.tracker.add_expense(amount, category, subcategory, description)
            self.clear_input_fields()
            self.view_expenses()  # Update the Treeview
            messagebox.showinfo("Success", "Expense added successfully!")
        
        except ValueError as e:
            messagebox.showerror("Invalid Input", f"Error: {e}")

    def clear_input_fields(self):
        """Clear the input fields after adding an expense."""
        self.entry_amount.delete(0, tk.END)  # Clear the amount entry field
        self.selected_category.set(list(self.tracker.categories.keys())[0])  # Reset category to default value
        self.selected_subcategory.set("")  # Reset subcategory
        self.entry_description.delete(0, tk.END)  # Clear the description entry field

    def view_expenses(self):
        """View all recorded expenses in the Treeview."""
        for row in self.tree.get_children():
            self.tree.delete(row)  # Clear previous data in the Treeview

        for index, row in self.tracker.df.iterrows():
            # Format the date to be displayed in the Treeview
            self.tree.insert("", "end", values=(row["Date"].strftime("%Y-%m-%d %H:%M:%S"), row["Amount"], row["Category"], row["Subcategory"], row["Description"]))

    def generate_report(self):
        """Generate a report and display it in a messagebox."""
        total_expenses, category_report, monthly_summary = self.tracker.generate_report()

        # Prepare the report text
        report_text = f"Total Expenses: {total_expenses}\n\n"
        report_text += "Expenses by Category and Subcategory:\n"
        for _, row in category_report.iterrows():
            report_text += f"{row['Category']} - {row['Subcategory']}: {row['Amount']}\n"

        report_text += "\nMonthly Summary:\n"
        for _, row in monthly_summary.iterrows():
            report_text += f"{row['Date']} - {row['Amount']}\n"

        messagebox.showinfo("Expense Report", report_text)


# Main program
if __name__ == "__main__":
    tracker = ExpenseTracker()
    root = tk.Tk()
    gui = ExpenseTrackerGUI(root, tracker)
    root.mainloop()
